# KNN
## -> with removing stopwords, without stemming
- Accuracy: 0.676
- F1-value:
- Roc-curve:

In [2]:
import pandas as pd
df = pd.read_csv('dataset.csv')
df = df.sample(10000)
df_target = df['humor']
df_data = df.copy()
df_data.drop(columns='humor')

df_target.head()

186255     True
60525      True
100563     True
50914     False
175519     True
Name: humor, dtype: bool

Preprocessing

In [3]:
from sklearn import preprocessing

#encode target to numeric
label_encoder = preprocessing.LabelEncoder()
df_target = label_encoder.fit_transform(df_target)
#df_target

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

#Apply CountVectorizer on text column

vectorizer = CountVectorizer(binary=True)

#If this vectorizer is used, a word which occurs multiple times is counted and not just displayed as 1 = word occurs 
#cVectorizer = CountVectorizer(binary=False)

#At the moment I am not able to apply the vectorizer on the whole dataset. To many records & less available RAM
matrix = vectorizer.fit_transform(df_data['text'].head(15))

print("Vectorizer created {} features.".format(len(vectorizer.get_feature_names())))

df_data_countVectorized = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())
#display(df_data_countVectorized)

Vectorizer created 131 features.


C:\Users\mcl.NB-MCL\anaconda3\envs\datamining\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re, string

#when running for the first time you need to activate this line for once.
#nltk.download('stopwords')

def tokenize(text):
    my_stopwords = set(stopwords.words('english'))
    df_without_stopwords = []
    
    #definition of stemming function
    token_pattern = re.compile(r"(?u)\b\w\w+\b") # split on whitespace
    
    tokens = token_pattern.findall(text)
    for item in tokens:
        if item not in my_stopwords:
            df_without_stopwords.append(item)
    return df_without_stopwords

#I'm not sure if we agreed to use stemming or not. I'll store the results in another df
#Stopwords are removed here as well
vectorizer = TfidfVectorizer(tokenizer=tokenize, min_df=0.0015)
matrix = vectorizer.fit_transform(df_data['text'])

df_data_without_stopwords = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())
#display(df_data_without_stopwords)

KNN

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Create train/test split
df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    df_data_without_stopwords, df_target, test_size=0.2, random_state=42)

#dt = DecisionTreeClassifier()

for n_neighbors in range(10,21):#[1,2,3,4,5]:#1,2,3,4,5
    knn_estimator = KNeighborsClassifier(n_neighbors)
    knn_estimator.fit(df_data_train, df_target_train)
    df_prediction = knn_estimator.predict(df_data_test)
    print("k= {} acc: {}".format(n_neighbors, accuracy_score(df_target_test, df_prediction)))


k= 10 acc: 0.6645
k= 11 acc: 0.676
k= 12 acc: 0.667
k= 13 acc: 0.671
k= 14 acc: 0.656
k= 15 acc: 0.6655
k= 16 acc: 0.648
k= 17 acc: 0.6565
k= 18 acc: 0.6505
k= 19 acc: 0.6525
k= 20 acc: 0.6485


In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()
    
cnf_matrix = confusion_matrix(df_target_test, df_prediction)
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix, classes=label_encoder.classes_)